In [1]:
from src.Tpm import *
from src.TpmTypes import *
mytpm = Tpm(useSimulator = True, host = '127.0.0.1', port = 2321)
mytpm.connect()

TSS.Py::__INIT__.PY invoked


In [3]:
mytpm.close()

In [3]:
primaryHandle = TPM_HANDLE(TPM_RH.OWNER)
# inSensitive = TPMS_SENSITIVE_CREATE(userAuth = b'This is my auth', data = b'This is my data')
inSensitive = TPMS_SENSITIVE_CREATE(userAuth = bytes([]), data = bytes([]))
# sym = TPMT_SYM_DEF_OBJECT(algorithm = TPM_ALG_ID.AES, keyBits = 128, mode = TPM_ALG_ID.CFB)
sym = TPMT_SYM_DEF_OBJECT(algorithm = TPM_ALG_ID.NULL, keyBits = 0, mode = TPM_ALG_ID.NULL) 
rsa = TPMS_RSA_PARMS(symmetric = sym, scheme = TPMS_ENC_SCHEME_OAEP(hashAlg=TPM_ALG_ID.SHA), keyBits = 2048, exponent = 65537)
# ecc = TPMS_ECC_PARMS(symmetric = sym, scheme = TPMS_SIG_SCHEME_ECDSA(hashAlg = None), curveID = TPM_ECC_CURVE.NIST_P256, kdf = TPM_ALG_ID.NULL)
# inPublic = TPMT_PUBLIC_PARMS(parameters = TPMS_SYMCIPHER_PARMS(sym = sym))
buffer = bytes(256)
inPublic = TPMT_PUBLIC(nameAlg = TPM_ALG_ID.SHA256, objectAttributes = TPMA_OBJECT.fixedTPM | TPMA_OBJECT.fixedParent | TPMA_OBJECT.userWithAuth | TPMA_OBJECT.sensitiveDataOrigin | TPMA_OBJECT.decrypt, authPolicy = None, parameters = rsa, unique = TPM2B_PUBLIC_KEY_RSA(buffer=None))
outsideInfo = bytes.fromhex("")
creationPCR = TPMS_PCR_SELECTION(hash = TPM_ALG_ID.SHA256, pcrSelect = bytes.fromhex("00"))
mytpm.allowErrors()
ret_primary = mytpm.CreatePrimary(primaryHandle, inSensitive, inPublic, outsideInfo, [])

80020000004100000131400000010000000940000009000001000000040000000000180001000b0002007200000010001700040800000100010000000000000000
Command                                   .                                                                   
TPMI_ST_COMMAND_TAG                       |   .tag                                       8002                 TPMI_ST_COMMAND_TAG.SESSIONS
UINT32                                    |   .commandSize                               00000041             65
TPM_CC                                    |   .commandCode                               00000131             TPM_CC.CreatePrimary
TPMS_COMMAND_HANDLES_CREATE_PRIMARY       |   .handles                                                        
TPMI_RH_HIERARCHY                         |   |   .primaryHandle                         40000001             TPM_RH.OWNER
UINT32                                    |   .authSize                                  00000009             9
TPMS_AUTH_COMMAND            

In [6]:
mytpm.EvictControl(auth=TPM_HANDLE.OWNER, objectHandle=ret_primary.handle, persistentHandle=TPM_HANDLE(0x81000000))

8002000000230000012040000001800000000000000940000009000001000081000000
Command                                   .                                                                   
TPMI_ST_COMMAND_TAG                       |   .tag                                       8002                 TPMI_ST_COMMAND_TAG.SESSIONS
UINT32                                    |   .commandSize                               00000023             35
TPM_CC                                    |   .commandCode                               00000120             TPM_CC.EvictControl
TPMS_COMMAND_HANDLES_EVICT_CONTROL        |   .handles                                                        
TPMI_RH_PROVISION                         |   |   .auth                                  40000001             TPM_RH.OWNER
TPMI_DH_OBJECT                            |   |   .objectHandle                          80000000             TPM_HR.TRANSIENT.000000
UINT32                                    |   .authSize             

In [5]:
mytpm.GetCapability(capability=TPM_CAP.HANDLES, property=0x81000000, propertyCount=0xFFFFFFFF)

8001000000160000017a0000000181000000ffffffff
Command                                   .                                                                   
TPMI_ST_COMMAND_TAG                       |   .tag                                       8001                 TPMI_ST_COMMAND_TAG.NO_SESSIONS
UINT32                                    |   .commandSize                               00000016             22
TPM_CC                                    |   .commandCode                               0000017a             TPM_CC.GetCapability
TPMS_COMMAND_HANDLES_GET_CAPABILITY       |   .handles                                                        
TPMS_COMMAND_PARAMS_GET_CAPABILITY        |   .parameters                                                     
TPM_CAP                                   |   |   .capability                            00000001             TPM_CAP.HANDLES
UINT32                                    |   |   .property                              81000000             

In [19]:
# parentHandle = TPM_HANDLE(0x81000000)
parentHandle = ret_primary.handle
# inSensitive = TPMS_SENSITIVE_CREATE(userAuth = b'This is my auth', data = b'This is my data')
inSensitive = TPMS_SENSITIVE_CREATE(userAuth = bytes([]), data = bytes([]))
# sym = TPMT_SYM_DEF_OBJECT(algorithm = TPM_ALG_ID.AES, keyBits = 128, mode = TPM_ALG_ID.CFB)
sym = TPMT_SYM_DEF_OBJECT(algorithm = TPM_ALG_ID.NULL, keyBits = 0, mode = TPM_ALG_ID.NULL) 
rsa = TPMS_RSA_PARMS(symmetric = sym, scheme = TPMS_NULL_ASYM_SCHEME(), keyBits = 2048, exponent = 65537)
# ecc = TPMS_ECC_PARMS(symmetric = sym, scheme = TPMS_SIG_SCHEME_ECDSA(hashAlg = None), curveID = TPM_ECC_CURVE.NIST_P256, kdf = TPM_ALG_ID.NULL)
# inPublic = TPMT_PUBLIC_PARMS(parameters = TPMS_SYMCIPHER_PARMS(sym = sym))
buffer = bytes(256)
inPublic = TPMT_PUBLIC(nameAlg = TPM_ALG_ID.SHA256, objectAttributes = TPMA_OBJECT.userWithAuth | TPMA_OBJECT.sensitiveDataOrigin | TPMA_OBJECT.decrypt | TPMA_OBJECT.sign, authPolicy = None, parameters = rsa, unique = TPM2B_PUBLIC_KEY_RSA(buffer=None))
outsideInfo = bytes.fromhex("")
creationPCR = TPMS_PCR_SELECTION(hash = TPM_ALG_ID.SHA256, pcrSelect = bytes.fromhex("00"))
mytpm.allowErrors()
ret_create = mytpm.Create(parentHandle, inSensitive, inPublic, outsideInfo, [])

80020000003f00000153800000000000000940000009000001000000040000000000160001000b000600000000001000100800000100010000000000000000
Command                                   .                                                                   
TPMI_ST_COMMAND_TAG                       |   .tag                                       8002                 TPMI_ST_COMMAND_TAG.SESSIONS
UINT32                                    |   .commandSize                               0000003f             63
TPM_CC                                    |   .commandCode                               00000153             TPM_CC.Create
TPMS_COMMAND_HANDLES_CREATE               |   .handles                                                        
TPMI_DH_OBJECT                            |   |   .parentHandle                          80000000             TPM_HR.TRANSIENT.000000
UINT32                                    |   .authSize                                  00000009             9
TPMS_AUTH_COMMAND            

128:0x0x80000000


AttributeError: 'NoneType' object has no attribute 'hex'